In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
data = pd.read_csv("./lending_club_loan_two.csv")
# data.head()
cols=['loan_amnt','term','int_rate','installment','emp_length','home_ownership','annual_inc','verification_status','loan_status','address','pub_rec']
data=data[cols]
data.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,address,pub_rec
0,10000.0,36 months,11.44,329.48,10+ years,RENT,117000.0,Not Verified,Fully Paid,"0174 Michelle Gateway\r\nMendozaberg, OK 22690",0.0
1,8000.0,36 months,11.99,265.68,4 years,MORTGAGE,65000.0,Not Verified,Fully Paid,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113",0.0
2,15600.0,36 months,10.49,506.97,< 1 year,RENT,43057.0,Source Verified,Fully Paid,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113",0.0
3,7200.0,36 months,6.49,220.65,6 years,RENT,54000.0,Not Verified,Fully Paid,"823 Reid Ford\r\nDelacruzside, MA 00813",0.0
4,24375.0,60 months,17.27,609.33,9 years,MORTGAGE,55000.0,Verified,Charged Off,"679 Luna Roads\r\nGreggshire, VA 11650",0.0


In [48]:
import re
colsX=['loan_amnt','term','int_rate','installment','emp_length','home_ownership','annual_inc','verification_status','pub_rec']
X=data[colsX]
Y=data['loan_status']
X['term'] = X['term'].str.split(' ').str[1].astype(int)
def extract_years(year_str):
    # Convert the input to a string if it's a float
    if isinstance(year_str, float):
        year_str = str(year_str)

    # Regular expression pattern to match numeric parts
    pattern = r'(\d+)'

    # Extract numeric part from the string
    match = re.search(pattern, year_str)
    if match:
        return int(match.group())
    elif '+' in year_str:
        # For "x+ years", extract x
        return int(year_str.split('+')[0])
    elif '<' in year_str:
        # For "< y years", extract y-1
        return int(year_str.split('<')[1]) - 1
    else:
        return None  # Unable to extract numeric part
X['emp_length'] = X['emp_length'].apply(extract_years)
X.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_33976\2131332335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['term'] = X['term'].str.split(' ').str[1].astype(int)
C:\Users\HP\AppData\Local\Temp\ipykernel_33976\2131332335.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['emp_length'] = X['emp_length'].apply(extract_years)


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,pub_rec
0,10000.0,36,11.44,329.48,10.0,RENT,117000.0,Not Verified,0.0
1,8000.0,36,11.99,265.68,4.0,MORTGAGE,65000.0,Not Verified,0.0
2,15600.0,36,10.49,506.97,1.0,RENT,43057.0,Source Verified,0.0
3,7200.0,36,6.49,220.65,6.0,RENT,54000.0,Not Verified,0.0
4,24375.0,60,17.27,609.33,9.0,MORTGAGE,55000.0,Verified,0.0


In [49]:
# Extracting pincode
# X['pin'] = X['address'].str.extract(r'(\d{5})$')
# X.drop('address', axis=1, inplace=True)

In [50]:
# unique_values = X['pin'].unique()
# unique_values

In [51]:
unique_values1=X['verification_status'].unique()
unique_values1


array(['Not Verified', 'Source Verified', 'Verified'], dtype=object)

In [52]:
mapping = {'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3, 'NONE': 4, 'ANY': 5}
X['home_ownership'] = X['home_ownership'].replace(mapping)

C:\Users\HP\AppData\Local\Temp\ipykernel_33976\1192662952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['home_ownership'] = X['home_ownership'].replace(mapping)


In [53]:
mapping = {'Not Verified': 0, 'Source Verified': 1, 'Verified': 1}
X['verification_status'] = X['verification_status'].replace(mapping)


C:\Users\HP\AppData\Local\Temp\ipykernel_33976\1070104094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['verification_status'] = X['verification_status'].replace(mapping)


In [55]:
Y.head()

0     Fully Paid
1     Fully Paid
2     Fully Paid
3     Fully Paid
4    Charged Off
Name: loan_status, dtype: object

In [56]:
mapping = {'Fully Paid': 1, 'Charged Off': 0}
Y=Y.replace(mapping)
# Y.head().unique()
Y.head()

0    1
1    1
2    1
3    1
4    0
Name: loan_status, dtype: int64

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model Training
xgb_classifier = xgb.XGBClassifier()
xgb_classifier.fit(X_train, y_train)

# Model Evaluation
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Deployment (Serialize and save the model if desired)



Accuracy: 0.8029063454788778


In [60]:
import joblib
joblib.dump(xgb_classifier, "xgboost_model.pkl")

['xgboost_model.pkl']

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model Training
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Model Evaluation
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy:", accuracy)






Epoch 1/10


9901/9901 [==============================] - 21s 2ms/step - loss: 0.5081 - accuracy: 0.8039
Epoch 2/10
9901/9901 [==============================] - 18s 2ms/step - loss: 0.4948 - accuracy: 0.8040
Epoch 3/10
9901/9901 [==============================] - 17s 2ms/step - loss: 0.4949 - accuracy: 0.8040
Epoch 4/10
9901/9901 [==============================] - 15s 2ms/step - loss: 0.4949 - accuracy: 0.8040
Epoch 5/10
9901/9901 [==============================] - 15s 2ms/step - loss: 0.4948 - accuracy: 0.8040
Epoch 6/10
9901/9901 [==============================] - 15s 2ms/step - loss: 0.4949 - accuracy: 0.8040
Epoch 7/10
9901/9901 [==============================] - 17s 2ms/step - loss: 0.4948 - accuracy: 0.8040
Epoch 8/10
9901/9901 [==============================] - 15s 2ms/step - loss: 0.4948 - accuracy: 0.8040
Epoch 9/10
9901/9901 [==============================] - 18s 2ms/step - loss: 0.4948 - accuracy: 0.8040
Epoch 10/10
9901/9901 [==============================] - 16s 2ms/ste

In [59]:
# Deployment (Save the model if desired)
model.save("ann_model")

INFO:tensorflow:Assets written to: ann_model\assets


INFO:tensorflow:Assets written to: ann_model\assets
